# Deep Learning w/ Python

## Home Assignment II. by Kristof Rabay - CNN, Reinforcement Learning

---

## A) Extend NN

Extend the NeuralNetwork class implementation with optional regularization and an alternative ADAM solver.

## Solution

1. Re-import existing class
2. Add regularization & ADAM optimizer

Will be learning the truth table of the `XOR` logical operator:

A | B | output |
--|---|--------|
0 | 0 | -1     |
0 | 1 | 1     |
1 | 0 | 1     |
1 | 1 | -1     |

### A.1 Redefining class

In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def plot_results_with_hyperplane(X, y, clf, clf_name=None, ax=None):
    df = pd.DataFrame(data=X, columns=['x', 'y'])
    df['label'] = y
    
    x_min, x_max = df.x.min() - .5, df.x.max() + .5
    y_min, y_max = df.y.min() - .5, df.y.max() + .5

    xx, yy = np.meshgrid(np.arange(x_min, x_max, .02), np.arange(y_min, y_max, .02))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    if ax is None:
        fig, ax = plt.subplots()
    
    ax.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired)
    ax.scatter(df.x, df.y, c=df.label, edgecolors='k')
    
    if clf_name is not None:
        ax.set_title(clf_name)
    
    return fig

In [116]:
def sigmoid(x):
    return 1. / (1. + np.exp(-x))

def sigmoid_prime(x):
    return sigmoid(x) * (1 - sigmoid(x))

def tanh(x):
    return np.tanh(x)

def tanh_prime(x):
    return 1. - np.tanh(x) ** 2

activation_function = {'sigmoid': {'f': sigmoid, "f'": sigmoid_prime},
                       'tanh': {'f': tanh, "f'": tanh_prime}}


def random_weight(layers, index):

    input_size = layers[index - 1] + 1
    
    extra_bias = int(not index == len(layers) - 1) # + 1 bias if not output layer 
    neuron_count = layers[index] + extra_bias
    
    shape = (input_size, neuron_count)

    return 2 * np.random.random(shape) - 1


def add_bias(X):
    if X.ndim == 1:
        return np.concatenate(([1], X))
    
    nrows, _ = X.shape
    ones = np.ones((nrows, 1))
    return np.concatenate((ones, X), axis=1)


class NeuralNetwork:
    
    def __init__(self, layers=[2, 2, 1], 
                 activation='sigmoid', 
                 alpha=0.1,
                 b1 = 0.9, b2 = 0.999, eps = 1e-08):
        
        self.activation = activation_function[activation]["f"]
        self.activation_prime = activation_function[activation]["f'"]
        
        self.alpha = alpha
        self.b1 = b1
        self.b2 = b2
        self.eps = eps

        self.layers = layers
        self.weights = [random_weight(layers, i) 
                        for i in range(1, len(layers))]
    
    def __str__(self):
        layers = " x ".join(str(l) for l in self.layers)
        return f'NeuralNet[{layers}]'
        
    def forward(self, x):
        nlayers = len(self.weights)

        a = [add_bias(x)]

        for layer in range(nlayers):
            dot_value = np.dot(a[layer], self.weights[layer])
            activation = self.activation(dot_value)
            a.append(activation)
        
        return a
    

    def delta(self, a, y):
        nlayers = len(self.weights)
        
        error = y - a[-1]
        deltas = [error * self.activation_prime(a[-1])]

        for layer in range(nlayers - 1, 0, -1):
            dot_value = np.dot(deltas[-1], self.weights[layer].T)
            delta = dot_value * self.activation_prime(a[layer])
            deltas.append(delta)

        deltas.reverse()
        return deltas 
    
    
        
    def backward(self, a, deltas):
        
        nlayers = len(self.weights)
        
        V_delta = 0
        S_delta = 0
        t = 0

        for layer in range(nlayers):
            
            inputs = np.atleast_2d(a[layer]).T
            delta = np.atleast_2d(deltas[layer])
            
            V_delta = self.b1 * V_delta + (1 - self.b1) * delta 
            S_delta = self.b2 * S_delta + (1 - self.b2) * delta * delta 
            
            V_delta_cor = V_delta / (1 - (self.b1 ** t))
            S_delta_cor = S_delta / (1 - (self.b2 ** t))
            
            self.weights[layer] = self.weights[layer] + self.alpha * ( (V_delta_cor) / ( (S_delta_cor ** (1/2) ) + self.eps) )
            

        
    def fit(self, X, y, epochs=100):
        nrows, nfeats = X.shape
        nlayers = len(self.weights)
            
        for iteration in range(epochs):
            for i in range(nrows):
                a = self.forward(X[i])
                deltas = self.delta(a, y[i])
                self.backward(a, deltas)
        
        return self
            
    def predict(self, X): 
        a = add_bias(X)
        for layer in self.weights:
            a = self.activation(np.dot(a, layer))
        return a > 0
    

### A.2 Fitting net to XOR data

In [117]:
inputs = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
labels = np.array([-1, 1, 1, -1])

In [121]:
nnet = NeuralNetwork(activation='tanh', alpha = 0.1, layers = [2, 2, 1], )

nnet.fit(inputs, labels, epochs = 3000)

plot_results_with_hyperplane(inputs, labels, nnet, str(nnet));

C:\Users\Krisz\Anaconda3\envs\ceu_dl\lib\site-packages\ipykernel_launcher.py:106: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\Krisz\Anaconda3\envs\ceu_dl\lib\site-packages\ipykernel_launcher.py:107: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\Krisz\Anaconda3\envs\ceu_dl\lib\site-packages\ipykernel_launcher.py:109: RuntimeWarning: invalid value encountered in true_divide
C:\Users\Krisz\Anaconda3\envs\ceu_dl\lib\site-packages\ipykernel_launcher.py:129: RuntimeWarning: invalid value encountered in greater


ValueError: cannot reshape array of size 30000 into shape (100,100)